In [1]:
import os
import sys 
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
from datetime import timedelta

import time
import pytz
import math
import locale

import openpyxl
from openpyxl import load_workbook

# import hvplot.pandas
# import plotly.graph_objects as go
# import matplotlib.pyplot as plt

import talib
from pybit import HTTP
# import requests 
# import json 

#pd.show_versions()

### Set environment variables from the .env in the local environment

In [2]:
load_dotenv()
my_api_key = os.getenv("API_KEY")
my_api_secret = os.getenv("API_SECRET")

# print(my_api_key)
# print(my_api_secret)

### Change Timezone if Needed

In [3]:
## my_list simply unpacks the list elements and pass each one of them as parameters to the print function  
# print(*pytz.all_timezones, sep='\n')

# In windows command prompt try:
#     This gives current timezone: tzutil /g
#     This gives a list of timezones: tzutil /l
#     This will set the timezone: tzutil /s "Central America Standard Time"


#os.system('tzutil /s "Eastern Standard Time"')  
#os.system('tzutil /s "Singapore Standard Time"')
#time.strftime('%Y-%m-%d %H:%m') 

### Functions Used to Pull Data from ByBit and Write Data to Files

In [4]:
# Adjust from_time to include prior 200 entries for that interval for ema200
def adjust_from_time(from_time, interval):
    
    delta = 200
    
    # Possible Values: 1 3 5 15 30 60 120 240 360 720 "D" "W" 
    if interval not in [1, 3, 5, 15, 30, 60, 120, 240, 360, 720, "D", "W"]:
        return from_time
    
    if interval == 'W':
        from_time = from_time - timedelta(weeks=delta)
    elif interval == 'D':
        from_time = from_time - timedelta(days=delta)
    else:
        from_time = from_time - timedelta(minutes=interval*delta)
    return from_time
        
def build_filename(params, extension):
    from_str = params['From_Time'].strftime('%Y-%m-%d')
    to_str = params['To_Time'].strftime('%Y-%m-%d')
    filename = f'{params["Symbol"]}_{from_str}_to_{to_str}_[{params["Interval"]}]{extension}'
    return filename

def get_bybit_kline_data(params, write_to_file=True):
    
    # Unauthenticated
    #session_unauth = HTTP(endpoint='https://api.bybit.com')

    # Authenticated
    session_auth = HTTP(
        endpoint = 'https://api.bybit.com',
        api_key = my_api_key,
        api_secret = my_api_secret
    )
    
    # The issue with ByBit API is that you can get a maximum of 200 bars from it. 
    # So if you need to get data for a large portion of the time you have to call it multiple times.

    print(f'Fetching data from ByBit.') 
    df_list = []

    # Adjust from_time to add 200 additional prior entries for ema200
    adjusted_from_time = adjust_from_time(params['From_Time'], params['Interval'])

    last_datetime_stamp = adjusted_from_time.timestamp()
    to_time = params['To_Time'].timestamp() 

    while last_datetime_stamp < to_time:
        # print(f'Fetching next 200 lines fromTime: {last_datetime_stamp} < to_time: {to_time}')
        # print(f'Fetching next 200 lines fromTime: {dt.datetime.fromtimestamp(last_datetime_stamp)} < to_time: {dt.datetime.fromtimestamp(to_time)}')
        result = session_auth.query_kline(symbol=params['Symbol'], interval=params['Interval'], **{'from':last_datetime_stamp})['result']
        tmp_df = pd.DataFrame(result)

        if tmp_df is None or (len(tmp_df.index) == 0):
            break

        tmp_df.index = [dt.datetime.fromtimestamp(x) for x in tmp_df.open_time]
        #tmp_df.index = [dt.datetime.utcfromtimestamp(x) for x in tmp_df.open_time]
        df_list.append(tmp_df)
        last_datetime_stamp = float(max(tmp_df.open_time) + 1) # Add 1 sec to last data received

        #time.sleep(2) # Sleep for x seconds, to avoid being locked out

    df = pd.concat(df_list)

    # Drop rows that have a timestamp greater than to_time
    df = df[df.open_time <= int(params['To_Time'].timestamp())]
    
    # Write to file
    if write_to_file:
        global OUPUT_FILE_FORMAT
        global HISTORICAL_FILES_PATH
        if 'csv' in OUPUT_FILE_FORMAT:
            fname = HISTORICAL_FILES_PATH + '\\' + build_filename(params, '.csv')
            df.to_csv (fname, index = True, header=True)
            print(f'Data written to file => [{fname}]')

        if 'xlsx' in OUPUT_FILE_FORMAT:
            fname = HISTORICAL_FILES_PATH + '\\' + build_filename(params, '.xlsx')
            df.to_excel (fname, index = True, header=True)
            print(f'Data written to file => [{fname}]')
    
    return df

def convertExcelToDataFrame(filename):
    wb = load_workbook(filename)
    ws = wb['Sheet1']

    # To convert a worksheet to a Dataframe you can use the values property. 
    # This is very easy if the worksheet has no headers or indices:
    # df = DataFrame(ws.values)

    # https://openpyxl.readthedocs.io/en/stable/pandas.html

    # If the worksheet does have headers or indices, such as one created by Pandas, 
    # then a little more work is required:
    from itertools import islice
    data = ws.values
    cols = next(data)[1:]
    data = list(data)
    idx = [r[0] for r in data]
    data = (islice(r, 1, None) for r in data)
    df = pd.DataFrame(data, index=idx, columns=cols)

    return df

### Calculate the Indicators and Signals

In [5]:
# ----------------------------------------------------------------------
# Function used determine trade entries (long/short)
# ----------------------------------------------------------------------
def trade_entries(open, ema200, macdsignal, cross):
    if open >= ema200 and macdsignal < 0 and cross == -1:
        return "Enter Long"
    elif open < ema200 and macdsignal > 0 and cross == 1:
        return "Enter Short"
    return None

# ----------------------------------------------------------------------
# Calculate and add indicators and signals to the DataFrame
# ----------------------------------------------------------------------
def add_indicators_and_signals(df):
    print('Adding indicators and Signals to Data.')
    
    # Set proper data types
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    df['volume'] = df['volume'].astype(np.int64)
    
    # Keep only this list of columns, delete all other columns
    final_table_columns = ['symbol', 'interval', 'open', 'high', 'low', 'close', 'volume']
    df = df[df.columns.intersection(final_table_columns)]
    #df.filter(final_table_columns)
    
    ## MACD - Moving Average Convergence/Divergence
    tmp = pd.DataFrame()
    tmp['macd'], tmp['macdsignal'], tmp['macdhist'] = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    tmp.drop(['macdhist'], axis = 1, inplace=True)
    df = df.join(tmp, rsuffix='_right')

    ## EMA - Exponential Moving Average
    df['ema200'] = talib.EMA(df['close'], timeperiod=200)

    # # Remove nulls
    df.dropna(inplace=True)

    # # Check if price is greater than ema200
    df['GT_ema200'] = np.where(df['open'] > df['ema200'], 'Bull', 'Bear')

    # macdsignal over macd then 1, under 0
    df['signal_over_under'] = np.where(df['macdsignal'] >= df['macd'], 1, 0)

    # macdsignal crosses macd
    df['cross'] = df['signal_over_under'].diff()

    # Remove nulls
    df.dropna(inplace=True)

    # Enter trade on the same candle as the crossing <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    #df['trade_status'] = df.apply(lambda x: trade_entries(x['open'], x['ema200'], x['macdsignal'], x['cross']), axis=1)
    
    # Enter trade in the candle after the crossing <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    df['trade_status'] = df.apply(lambda x: trade_entries(x['open'], x['ema200'], x['macdsignal'], x['cross']), axis=1).shift(1)

    # Add and Initialize to 0 new columns for take_profit/stop_loss and profits/loss
    df['take_profit'] = 0.0
    df['stop_loss'] = 0.0
    df['win'] = 0.0
    df['loss'] = 0.0
    df['fee'] = 0.0

    return df

### Based on Indicators Create Trades and Statistics

In [6]:
# ----------------------------------------------------------------------
# Print Statistics
# ----------------------------------------------------------------------
def print_trade_stats(total_wins, total_losses, nb_wins, nb_losses, total_fees_paid, 
                      max_conseq_wins, max_conseq_losses, min_win_loose_index, max_win_loose_index):
    total_trades = nb_wins + nb_losses
    success_rate = (nb_wins / total_trades * 100) if total_trades != 0 else 0
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
    print(f'\n-------------------- Statistics --------------------')
    print(f'Winning Trades: {nb_wins}')
    print(f'Max # Consecutive Wins: {max_conseq_wins}')
    print('---')
    print(f'Losing Trades: {nb_losses}')
    print(f'Max # Consecutive Losses: {max_conseq_losses}')
    print('---')
    print(f'Total Trades: {total_trades}')
    print(f'Success Rate: {success_rate:.1f}%')
    print(f'Win/Loose Index: Min[{min_win_loose_index}] Max[{max_win_loose_index}]')
    print()
    print(f'Total Wins: {locale.currency(total_wins, grouping=True)}');
    print(f'Total Losses: {locale.currency(total_losses, grouping=True)}');
    print(f'Total Fees Paid: {locale.currency(total_fees_paid, grouping=True)}')
    print(f'Total P/L: {locale.currency(total_wins+total_losses-total_fees_paid, grouping=True)}\n')
    #print('-------------------------------------------------------')
    
def determine_win_or_loose(row):
    if row['win'] != 0:
        return 'W'
    elif row['loss'] != 0:
        return 'L'
    else:
        return None
  
     
# Returns 4 values.  
# 1) Maximum number of consecutive win trades within the date range
# 2) Maximum number of consecutive loss trades within the date range
# 3) Minimum loosing index
# 4) Maximum loosing index
def analyze_win_lose(df):
    # Part 1: Max consecutive wins or losses
    df['W/L'] = df.apply(determine_win_or_loose, axis=1)
    values = df['W/L'].values.tolist()
    values = list(filter(None, values)) # Remove nulls

    last_index = len(values)-1
    count_W = 0
    count_L = 0
    max_W = 0
    max_L = 0

    for i, val in enumerate(values):
        if val == 'W':
            count_W += 1
            if i == last_index and count_W > max_W:
                max_W = count_W
            elif i < last_index and values[i+1] != val:
                    if count_W > max_W:
                        max_W = count_W
                    count_W = 0
        elif val == 'L':
            count_L += 1
            if i == last_index and count_L > max_L:
                max_L = count_L
            elif i < last_index and values[i+1] != val:
                    if count_L > max_L:
                        max_L = count_L
                    count_L = 0
        #print(f'Index[{i}] Value[{val}] - count_W: {count_W}, count_L: {count_L}, max_W: {max_W}, max_L: {max_L}')
        
    # Part 2: Loosing Metric
    win_loose_index = 0
    min_win_loose_index = 0
    max_win_loose_index = 0
    
    for i, val in enumerate(values):
        if val == 'W':
            win_loose_index += 1
        elif val == 'L':
            win_loose_index -= 1
        
        if win_loose_index > max_win_loose_index:
            max_win_loose_index = win_loose_index
        elif win_loose_index < min_win_loose_index:
            min_win_loose_index = win_loose_index
            
        #print(f'[{i}][{val}]: current[{win_loose_index}] min[{min_win_loose_index}] max[{max_win_loose_index}]')
        
    return max_W, max_L, min_win_loose_index, max_win_loose_index


# ----------------------------------------------------------------------------
# Process Trades: Add Trades to the Dataframe, write results to Excel files
# ----------------------------------------------------------------------------
def process_trades(df, params):
    print('Processing Trades.')
    nb_wins = 0
    nb_losses = 0

    position_size = 0.0
    entry_price = 0.0
    stop_loss = 0.0
    take_profit = 0.0

    trade_status = ''
    total_wins = 0.0
    total_losses = 0.0
    total_fees_paid = 0.0

    # We use numeric indexing to update values in the DataFrame
    # Find the column indexes
    trade_status_col_index = df.columns.get_loc("trade_status")
    tp_col_index = df.columns.get_loc("take_profit")
    sl_col_index = df.columns.get_loc("stop_loss")
    wins_col_index = df.columns.get_loc("win")
    losses_col_index = df.columns.get_loc("loss")
    fee_col_index = df.columns.get_loc("fee")

    TP_PCT = params['Take_Profit_PCT'] / 100
    SL_PCT = params['Stop_Loss_PCT'] / 100
    MAKER_FEE_PCT = params['Maker_Fee_PCT'] / 100
    TAKER_FEE_PCT = params['Taker_Fee_PCT'] / 100

    for i, row in enumerate(df.itertuples(index=True), 0):

        # ------------------------------- Longs -------------------------------
        if trade_status == '' and row.trade_status == 'Enter Long':
            
            entry_price = row.open
            stop_loss = entry_price - (SL_PCT * entry_price)
            take_profit = entry_price + (TP_PCT * entry_price)
            df.iloc[i, tp_col_index] = take_profit
            df.iloc[i, sl_col_index] = stop_loss
            # Entry Fee
            entry_fee = params['Trade_Amount'] * TAKER_FEE_PCT
            df.iloc[i, fee_col_index] += entry_fee
            total_fees_paid += entry_fee
            
            # We exit in the same candle we entered, hit stop loss
            if row.low <= stop_loss:
                loss = params['Trade_Amount'] * SL_PCT * -1
                df.iloc[i, trade_status_col_index] = 'Enter/Exit Long'
                df.iloc[i, losses_col_index] = loss
                total_losses += loss
                trade_status = ''
                nb_losses += 1
                #Exit Fee 'loss'
                exit_fee = (params['Trade_Amount'] - loss) * TAKER_FEE_PCT
                df.iloc[i, fee_col_index] += exit_fee
                total_fees_paid += exit_fee
                
            # We exit in the same candle we entered, take profit
            elif row.high >= take_profit:
                win = params['Trade_Amount'] * TP_PCT
                df.iloc[i, trade_status_col_index] = 'Enter/Exit Long'
                df.iloc[i, wins_col_index] = win
                total_wins += win
                trade_status = ''
                nb_wins += 1
                #Exit Fee 'win'
                exit_fee = (params['Trade_Amount'] + win) * MAKER_FEE_PCT
                df.iloc[i, fee_col_index] += exit_fee
                total_fees_paid += exit_fee
                
            # We just entered 'Enter long' in this candle so set the status to 'Long'
            else:
                trade_status = 'Long'

        elif trade_status in ['Long'] and pd.isnull(row.trade_status):
            if row.low <= stop_loss:
                loss = params['Trade_Amount'] * SL_PCT * -1
                df.iloc[i, trade_status_col_index] = 'Exit Long'
                df.iloc[i, losses_col_index] = loss
                df.iloc[i, tp_col_index] = take_profit
                df.iloc[i, sl_col_index] = stop_loss
                total_losses += loss
                trade_status = ''
                nb_losses += 1
                # Exit Fee 'loss'
                exit_fee = (params['Trade_Amount'] - loss) * TAKER_FEE_PCT
                df.iloc[i, fee_col_index] += exit_fee
                total_fees_paid += exit_fee
            elif row.high >= take_profit:
                win = params['Trade_Amount'] * TP_PCT
                df.iloc[i, trade_status_col_index] = 'Exit Long'
                df.iloc[i, wins_col_index] = win
                df.iloc[i, tp_col_index] = take_profit
                df.iloc[i, sl_col_index] = stop_loss
                total_wins += win
                trade_status = ''
                nb_wins += 1
                # Exit Fee 'win'
                exit_fee = (params['Trade_Amount'] + win) * MAKER_FEE_PCT
                df.iloc[i, fee_col_index] += exit_fee
                total_fees_paid += exit_fee
            else:
                df.iloc[i, trade_status_col_index] = 'Long'
                df.iloc[i, tp_col_index] = take_profit
                df.iloc[i, sl_col_index] = stop_loss
                trade_status = 'Long'

        elif trade_status in ['Long'] and row.trade_status in ['Enter Long', 'Enter Short']:
            # If we are in a long and encounter another 'Enter Long' or a 'Enter Short' signal,
            # ignore the signal and override the value with 'Long', we are already in a 'Long' trade
            df.iloc[i, trade_status_col_index] = 'Long'
            df.iloc[i, tp_col_index] = take_profit
            df.iloc[i, sl_col_index] = stop_loss

        # ------------------------------- Shorts ------------------------------- 
        elif trade_status == '' and row.trade_status == 'Enter Short':
            entry_price = row.open
            #position_size = (params['Trade_Amount'] / row.open) if row.open != 0 else 0
            stop_loss = entry_price + (SL_PCT * entry_price)
            take_profit = entry_price - (TP_PCT * entry_price)
            df.iloc[i, tp_col_index] = take_profit
            df.iloc[i, sl_col_index] = stop_loss
            # Entry Fee
            entry_fee = params['Trade_Amount'] * TAKER_FEE_PCT
            df.iloc[i, fee_col_index] += entry_fee
            total_fees_paid += entry_fee
                
             # We exit in the same candle we entered, hit stop loss
            if row.high >= stop_loss:
                loss = SL_PCT * params['Trade_Amount'] * -1
                df.iloc[i, trade_status_col_index] = 'Enter/Exit Short'
                df.iloc[i, losses_col_index] = loss
                total_losses += loss
                trade_status = ''
                nb_losses += 1
                # Exit Fee 'loss'
                exit_fee = (params['Trade_Amount'] + loss) * TAKER_FEE_PCT
                df.iloc[i, fee_col_index] += exit_fee
                total_fees_paid += exit_fee
             # We exit in the same candle we entered, hit take profit
            elif row.low <= take_profit:
                win = params['Trade_Amount'] * TP_PCT
                df.iloc[i, trade_status_col_index] = 'Enter/Exit Short'
                df.iloc[i, wins_col_index] = win
                total_wins += win
                trade_status = ''
                nb_wins += 1
                # Exit Fee 'loss'
                exit_fee = (params['Trade_Amount'] - win) * MAKER_FEE_PCT
                df.iloc[i, fee_col_index] += exit_fee
                total_fees_paid += exit_fee
            # We just entered 'Enter Short' in this candle, so set the status to 'Short'
            else:
                trade_status = 'Short'

        elif trade_status in ['Short'] and pd.isnull(row.trade_status):
            if row.high >= stop_loss:
                loss = SL_PCT * params['Trade_Amount'] * -1
                df.iloc[i, trade_status_col_index] = 'Exit Short'
                df.iloc[i, losses_col_index] = loss
                df.iloc[i, tp_col_index] = take_profit
                df.iloc[i, sl_col_index] = stop_loss
                total_losses += loss
                trade_status = ''
                nb_losses += 1
                # Exit Fee 'loss'
                exit_fee = (params['Trade_Amount'] + loss) * TAKER_FEE_PCT
                df.iloc[i, fee_col_index] += exit_fee
                total_fees_paid += exit_fee
            elif row.low <= take_profit:
                win = params['Trade_Amount'] * TP_PCT
                df.iloc[i, trade_status_col_index] = 'Exit Short'
                df.iloc[i, wins_col_index] = win
                df.iloc[i, tp_col_index] = take_profit
                df.iloc[i, sl_col_index] = stop_loss
                total_wins += win
                trade_status = ''
                nb_wins += 1
                # Exit Fee 'win'
                exit_fee = (params['Trade_Amount'] - win) * MAKER_FEE_PCT
                df.iloc[i, fee_col_index] += exit_fee
                total_fees_paid += exit_fee
            else:
                df.iloc[i, trade_status_col_index] = 'Short'
                df.iloc[i, tp_col_index] = take_profit
                df.iloc[i, sl_col_index] = stop_loss
                trade_status = 'Short'

        elif trade_status in ['Short'] and row.trade_status in ['Enter Long', 'Enter Short']:
            # If we are in a long and encounter another 'Enter Long' or a 'Enter Short' signal,
            # ignore the signal and override the value with 'Long', we are already in a 'Short' trade
            df.iloc[i, trade_status_col_index] = 'Short'
            df.iloc[i, tp_col_index] = take_profit
            df.iloc[i, sl_col_index] = stop_loss

    global OUPUT_FILE_FORMAT, RESULTS_PATH
    test_num = str(params['Test_Num']) + '_'
    if 'csv' in OUPUT_FILE_FORMAT:
        fname = RESULTS_PATH + f'\\{test_num}_' + build_filename(param, '_Trades.csv')
        df.to_csv (fname, index = True, header=True)
        print(f'Trades file created => [{fname}]')

    if 'xlsx' in params['OUPUT_FILE_FORMAT:
        fname = params['RESULTS_PATH'] + f'\\{test_num}_' + build_filename(params, '_Trades.xlsx')
        df.to_excel (fname, index = True, header=True)
        print(f'Trades file created => [{fname}]')
        
    max_conseq_wins, max_conseq_losses, min_win_loose_index, max_win_loose_index = analyze_win_lose(df)
    
    # print_trade_stats(
    #     total_wins, 
    #     total_losses, 
    #     nb_wins, 
    #     nb_losses, 
    #     total_fees_paid, 
    #     max_conseq_wins, 
    #     max_conseq_losses,  
    #     min_win_loose_index, 
    #     max_win_loose_index
    # )
    
    # Store results in Results DataFrame
    total_trades = nb_wins + nb_losses
    success_rate = (nb_wins / total_trades * 100) if total_trades != 0 else 0
    params['Results'] = params['Results'].append(
        {
            'Test #': params['Test_Num'], 
            'Symbol': params['Symbol'], 
            'From': params['From_Time'].strftime("%Y-%m-%d"),
            'To': params['To_Time'].strftime("%Y-%m-%d"),
            'Interval': params['Interval'], 
            'Amount': params['Trade_Amount'], 
            'TP %': params['Take_Profit_PCT'], 
            'SL %': params['Stop_Loss_PCT'], 
            'Maker Fee %': params['Maker_Fee_PCT'],
            'Taker Fee %': params['Taker_Fee_PCT'],
            
            'Wins': nb_wins, 
            'Losses': nb_losses, 
            'Total Trades': total_trades,
            'Success Rate': f'{success_rate:.1f}%',
            'Loss Idx': min_win_loose_index, 
            'Win Idx': max_win_loose_index, 
            'Wins $': total_wins, 
            'Losses $': total_losses,
            'Fees $': total_fees_paid, 
            'Total P/L': total_wins + total_losses - total_fees_paid
        }, 
        ignore_index=True, 
    )

    return df

In [7]:
# Print parameter values. 
# 'all'=True prints all values
# 'all'=False prints only relevant ones (default)
def print_parameters(params, all=False):
    if all:
        for key, value in params.items():
            print(f'{key}: {value}')
        print()
    else:
        print(f'SYMBOL: {params["Symbol"]}')
        print(f'FROM_TIME: {params["From_Time"]}')
        print(f'TO_TIME: {params["To_Time"]}')
        print(f'INTERVAL: {params["Interval"]}')
        print(f'TRADE_AMOUNT: {params["Trade_Amount"]}')
        print(f'TAKE_PROFIT_PCT: {params["Take_Profit_PCT"]}%')
        print(f'STOP_LOSS_PCT: {params["Stop_Loss_PCT"]}%')
        print(f'MAKER_FEE_PCT: {params["Maker_Fee_PCT"]}%')
        print(f'TAKER_FEE_PCT: {params["Taker_Fee_PCT"]}%')
        print('-------------------------------------------------------')
    
def validate_params(params):
    if not isinstance(params['From_Time'], dt.datetime):
        raise Exception(f'Invalid Parameter [FROM_TIME] = {params["From_Time"]}')
        
    if not isinstance(params['To_Time'], dt.datetime):
        raise Exception(f'Invalid Parameter [TO_TIME] = {params["To_Time"]}')
        
    if params["Interval"] not in ["1", "3", "5", "15", "30", "60", "120", "240", "360", "720", "D", "W"]:
        raise Exception(f'Invalid Parameter [INTERVAL] = {params["Interval"]}')
        
    trade_amount = params["Trade_Amount"]
    if not isinstance(trade_amount, float) or trade_amount <= 0:
        raise Exception(f'Invalid Parameter [TRADE_AMOUNT] = {trade_amount}. Must be a positive value of type float.')
        
    take_profit_pct = params["Take_Profit_PCT"]
    if not isinstance(take_profit_pct, float) or take_profit_pct <= 0:
        raise Exception(f'Invalid Parameter [TAKE_PROFIT_PCT] = {take_profit_pct}. Must be a positive value of type float.')
        
    stop_loss_pct = params["Stop_Loss_PCT"]
    if not isinstance(stop_loss_pct, float) or stop_loss_pct <= 0:
        raise Exception(f'Invalid Parameter [STOP_LOSS_PCT] = {stop_loss_pct}. Must be a positive value of type float.')
        
    maker_fee_pct = params["Maker_Fee_PCT"]
    if not isinstance(maker_fee_pct, float):
        raise Exception(f'Invalid Parameter [Maker_Fee_PCT] = {maker_fee_pct}. Must be a positive value of type float.')
        
    taker_fee_pct = params["Taker_Fee_PCT"]
    if not isinstance(taker_fee_pct, float):
        raise Exception(f'Invalid Parameter [Taker_Fee_PCT] = {taker_fee_pct}. Must be a positive value of type float.')
    
    # Convert all items to lower case
    global OUPUT_FILE_FORMAT
    formats_list = OUPUT_FILE_FORMAT
    if not isinstance(formats_list, list) or len(OUPUT_FILE_FORMAT) == 0:
        raise Exception(f'Invalid Global Setting [OUPUT_FILE_FORMAT] = {formats_list}.')
    OUPUT_FILE_FORMAT = [x.lower() for x in OUPUT_FILE_FORMAT]
   
def load_test_cases_from_file(filename):
    print(f'Loading test cases from file => [{filename}]')
    df = convertExcelToDataFrame(filename)
    df['Interval'] = df['Interval'].astype(str)
    df['Trade Amount'] = df['Trade Amount'].astype(float)
    df['TP %'] = df['TP %'].astype(float)
    df['SL %'] = df['SL %'].astype(float)
    df['Maker Fee %'] = df['Maker Fee %'].astype(float)
    df['Taker Fee %'] = df['Taker Fee %'].astype(float)
    return df

def create_empty_results_df():
    df = pd.DataFrame(columns=['Test #', 'Symbol', 'From', 'To', 'Interval', 'Amount', 'TP %', 'SL %', 'Maker Fee %', 'Taker Fee %',
                               'Wins', 'Losses', 'Total Trades', 'Success Rate', 'Loss Idx', 'Win Idx','Wins $', 'Losses $', 'Fees $', 
                               'Total P/L'])
    return df

# Run the backtesting
def backtest(params):
    
    print(f'----------------------- TEST #{params["Test_Num"]} -----------------------')
    #print_parameters(params)
    validate_params(params)

    # Method 1 (slow): Get historical data directly from the ByBit API
    # --------------------------------------------------------------------
    global MIN_DATA_SIZE
    df = get_bybit_kline_data (params)
    if df is None:
        print(f'\nNo data was returned from ByBit. Unable to backtest strategy.')
        raise Exception("No data returned by ByBit")
    elif len(df) <= MIN_DATA_SIZE:
        print(f'\nData rows = {len(df)}, less than MIN_DATA_SIZE={MIN_DATA_SIZE}. Unable to backtest strategy.')
        raise Exception("Unable to Run Strategy on Data Set")

    # Method 2 (fast): Get historical data from previously saved files
    # --------------------------------------------------------------------
    # filename = 'ByBit Historical\\BTCUSDT_2021-01-01_to_2021-11-27_30.xlsx'
    # print(f'Reading data from file => [{filename}]')
    # df = convertExcelToDataFrame(filename)

    df = add_indicators_and_signals(df);
    df = process_trades(df, params)
    return df

### Running the BackTesting

In [8]:
import warnings
warnings.filterwarnings('ignore')

# Global Application Settings
HISTORICAL_FILES_PATH = 'ByBit Historical' # Folder location where to store ByBit original raw data
TEST_CASES_FILE_PATH = 'TestCases.xlsx' # File comntaining test cases
RESULTS_PATH = 'BackTesting Results' # Folder location where to store the back testing output
OUPUT_FILE_FORMAT = ['xlsx']  # Prefered format(s) for the ouput: csv, xlsx or both. Ex: ['csv', 'xlsx']
MIN_DATA_SIZE = 201 # Cannot run Strategy on data set less than this value

# Load test cases from Excel file
test_cases_df = load_test_cases_from_file(TEST_CASES_FILE_PATH)
#test_cases_df

# Create DataFrame to store results
results_df = create_empty_results_df()
#results_df

# Run back test each test case
for index, row in test_cases_df.iterrows():
    params = {
        'Test_Num' : index
        , 'Symbol' : row.Symbol
        , 'From_Time' : row.From
        , 'To_Time' : row.To
        , 'Interval' : row.Interval 
        , 'Trade_Amount' : row['Trade Amount'] 
        , 'Take_Profit_PCT' : row['TP %']
        , 'Stop_Loss_PCT' : row['SL %']
        , 'Maker_Fee_PCT' : row['Maker Fee %']
        , 'Taker_Fee_PCT' : row['Taker Fee %']

        , 'Results' : results_df
    }
    df = backtest(params)
    results_df = params['Results']
    
# Save results to file
results_df = results_df.set_index('Test #')
if 'csv' in OUPUT_FILE_FORMAT:
    fname = RESULTS_PATH + '\\' + 'Statistics.csv'
    results_df.to_csv (fname, index = True, header=True)
    print(f'Stats file created => [{fname}]')

if 'xlsx' in OUPUT_FILE_FORMAT:
    fname = RESULTS_PATH + '\\' + 'Statistics.xlsx'
    results_df.to_excel (fname, index = True, header=True)
    print(f'Stats file created => [{fname}]')
    
# Display Results DataFrame in JupyerLab
results_df

# def main():
#     print("main()")

# if __name__ == "__main__":
#     main()

Loading test cases from file => [TestCases.xlsx]
----------------------- TEST #1 -----------------------
Fetching data from ByBit.
Data written to file => [ByBit Historical\BTCUSDT_2021-10-01_to_2021-11-30_[30].xlsx]
Adding indicators and Signals to Data.
Processing Trades.


KeyError: 'OUPUT_FILE_FORMAT'